In [ ]:
!pip install rasterio
!pip install geopandas

In [1]:
%matplotlib inline
import os
# from google.colab import drive
# drive.mount('/content/drive')
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'C:\Users\ruben\Desktop\GEM\geoloc2\Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:\Users\ruben\yolov7')
# from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime
from clasificacion_chinchetas import *

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 29 10:45:35 2023
@author: Alberto
"""
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from shapely.geometry import Polygon
import cv2
import time
from pathlib import Path
import torch
import tqdm
import numpy as np
from models.experimental import attempt_load
from utils.datasets import LoadImages , letterbox
from utils.general import check_img_size, non_max_suppression, scale_coords, xyxy2xywh, set_logging
from utils.torch_utils import select_device, time_synchronized, TracedModel
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import cv2
import torch,torchvision
import numpy as np
import torch.nn as nn
from PIL import Image
from torchvision import models
from torchsummary import summary
opt_img_size=256
class modelo():
    def __init__(self,weights=["yolov7.pt"]):
        """inicializa el modelo con los pesos"""
        self.device = torch.device(0 if torch.cuda.is_available() else "cpu")
        model = attempt_load(weights, map_location=self.device)
        model = TracedModel(model, self.device, opt_img_size)
        self.model= model
        
    def detect(self,opt_conf_thres,opt_source="train/images",display=False,imagen_s=np.array([1,1])):
        """Genera las deteccion de objetos por imagen precargada o por carpeta/archivo"""
        vector=[]
        opt_no_trace=False
        opt_iou_thres=0.45
        opt_save_conf=True
        opt_classes=None
        opt_agnostic_nms=False
        opt_augment=False
        opt_no_trace=False
        source,  imgsz, trace = opt_source, opt_img_size, not opt_no_trace
        set_logging()
        half = self.device.type != 'cpu'  # half precision only supported on CUDA
    # load FP32 model
        stride = int(self.model.stride.max())  # self.model stride
        imgsz = check_img_size(imgsz, s=stride)  # check img_size
        if half:
            self.model.half()  # to FP16
        names = self.model.module.names if hasattr(self.model, 'module') else self.model.names
        # Set Dataloader
        if imagen_s.shape[0]==2:
            dataset = LoadImages(source, img_size=imgsz, stride=stride)
            t0 = time.time()
            for path, img, im0s, vid_cap in tqdm.tqdm(dataset):
                img = torch.from_numpy(img).to(self.device)
                img = img.half() if half else img.float()  # uint8 to fp16/32
                img /= 255.0  # 0 - 255 to 0.0 - 1.0
                if img.ndimension() == 3:
                    img = img.unsqueeze(0)
                t1 = time_synchronized()
                with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
                    pred = self.model(img, augment=opt_augment)[0]
                t2 = time_synchronized()
                pred = non_max_suppression(pred, opt_conf_thres, opt_iou_thres, classes=opt_classes, agnostic=opt_agnostic_nms)
                t3 = time_synchronized()
                for i, det in enumerate(pred):  # detections per image
                    p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)
                    p = Path(p)  # to Path
                    gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                    if len(det):
                        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
                        for c in det[:, -1].unique():
                            n = (det[:, -1] == c).sum()  # detections per class
                            s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                        for *xyxy, conf, cls in reversed(det):
                            xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                            line = (cls, *xywh, conf) if opt_save_conf else (cls, *xywh)  # label format
                            vector.append(list(line)+[p.name[:-4]])  
                if display:
                    print(f'{s}Done. ({(1E3 * (t2 - t1)):.1f}ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS')
        else:
            img0=imagen_s  
            img = letterbox(img0, imgsz, stride)[0]
            img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
            img = np.ascontiguousarray(img)
            img = torch.from_numpy(img).to(self.device)
            img = img.half() if half else img.float()
            img /= 255.0  # 0 - 255 to 0.0 - 1.0
            if img.ndimension() == 3:
                img = img.unsqueeze(0)
            t1 = time_synchronized()
            with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
                pred = self.model(img, augment=opt_augment)[0]
                # pred_o=pred
            t2 = time_synchronized()
            pred = non_max_suppression(pred, opt_conf_thres, opt_iou_thres, classes=opt_classes, agnostic=opt_agnostic_nms)
            t3 = time_synchronized()
            for i, det in enumerate(pred):  # detections per image
                s, im0 =  '', img0
#                 p = Path(p)  # to Path
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                if len(det):
                    det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
                    for c in det[:, -1].unique():
                        n = (det[:, -1] == c).sum()  # detections per class
                        s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                    for *xyxy, conf, cls in reversed(det):
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if opt_save_conf else (cls, *xywh)  # label format
                        vector.append(line)
        return vector
    
def correct_orientation(img_rgb,dim,pattern_path="pattern1.png"):
    """Determina el angulo donde existe la mayor deteccion de lineas rectas"""
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    template = cv2.imread(pattern_path,0)
    w, h = template.shape[::-1]
    template  = cv2.resize(template,(int(w/4),int(h/4)))
    image_ro=img_gray.copy()
    angulo=0
    an=[]
    le=0
    angulo_f=0
    imagen_final=img_rgb.copy()
    for i in range(-45,45,1):
        angulo=i
        M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
        image_ro = cv2.warpAffine(img_gray, M, (dim,dim))
        res = cv2.matchTemplate(image_ro,template,cv2.TM_CCOEFF_NORMED)
        threshold =.5
        loc = np.where( res >= threshold)
        com=len(loc[0])
        if com>0:
            an.append(angulo)
            if le<com:
                le=com
                angulo_f=angulo
                M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
                imagen_final = cv2.warpAffine(img_rgb, M, (dim,dim))
    return angulo_f,imagen_final

def verificacion(im):
    """filtro para determinar cuanta area verde existe en la imagen"""
    hsv=cv2.cvtColor(im,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(hsv,(12,30,0),(160,232,160))
    verde=int((np.sum(mask)/im.shape[0]**2/255)*100)
    return verde 

def vector2xy(vector,w,h,dim=700,nameimg="image",angle=0):
    """Transforma el vector de deteccion en coordendas porcentuales y enteras de la imagen"""
    s=[]
    for v in vector:
        str_v=(str(v).replace("tensor(","").replace("=","").replace(", device","").replace("[","").replace("'cuda:0'","").replace("]","").replace(".)","").replace(")","").replace("(","").replace("']","").replace("'","").strip().split(","))
#         h,w=dim,dim
        x1 = int( float(str_v[1]) * w )
        y1 = int( float(str_v[2]) * h )
        xw = int( float(str_v[3]) * w /2)
        yw = int( float(str_v[4]) * h /2)
        start_point_im = ((x1 - xw), (y1 - yw))
        end_point_im   = ((x1 + xw), (y1 + yw))
        start_point_100 = ((x1 - xw)/w, (y1 - yw)/h)
        end_point_100   = ((x1 + xw)/w, (y1 + yw)/h)
        area=xw*yw
        conf=str_v[5]
        try:
            nameimg=str_v[6]
        except:
            pass
        if str(str_v[0])=="0":
            tipo="casa"
        else:
            tipo="terreno"
        if int(xw)!=0 and int(yw)!=0:# and (xw/yw<=3.2 and yw/xw<=3.2):
            s.append([tipo,start_point_im,end_point_im,start_point_100,end_point_100,area,conf,nameimg," ".join(str_v[:5])])
    df_cache=pd.DataFrame(s,columns=["Tipo","start_point_im","end_point_im","start_point_100","end_point_100","area","conf","imagen","vector_o"])
    df_cache.drop_duplicates().reset_index(drop=True,inplace=True)
    return df_cache
    
def imshow_detect(df_cache,imagen_n,nameimg="image"):
    """muestra la imagen con las detecciones"""
    for i in range(len(df_cache)):
            if df_cache["Tipo"][i]=="casa":
                x,y=df_cache["start_point_im"][i]
                cv2.rectangle(imagen_n,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,0,255),2)
#                 cv2.putText(imagen_n, str(df_cache["conf"][i]), (x+50,y+50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            else:
                x,y=df_cache["start_point_im"][i]
                cv2.rectangle(imagen_n,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,255,0),2)
#                 cv2.putText(imagen_n, str(int(float(df_cache["conf"][i])*100)/100),(x+50,y+50) , cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
    cv2.imshow(nameimg,imagen_n)
    cv2.waitKey()
    cv2.destroyAllWindows()

def rotacion_detect(startpoint,endpoint,angle,proyecciones,w,h,dim):
    """Rotata el cuadro detectado en el sistema de coordenadas inicial"""
    point1=np.min((proyecciones,proyecciones),axis=1)[0]
    min_y,min_x=point1[0],point1[1]
    point2=np.max((proyecciones,proyecciones),axis=1)[0]
    max_y,max_x=point2[0],point2[1]
    min_y,min_x,max_y,max_x,proyecciones
    tipos=["casa","terreno"]
    y1,x1=startpoint
    y2,x2=endpoint
    x1,y1=(x1)*2-1,(y1)*2-1
    x2,y2=(x2)*2-1,(y2)*2-1
    angle=angle*math.pi/180
    x1,y1=x1*(w/dim),y1*(h/dim)
    x2,y2=x2*(w/dim),y2*(h/dim)
    #x_p, y_p son los puntos de un rectangulo en el orden inverso al manecillas del reloj
    x1p=max_x-((x1*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y1p=min_y+((x1*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x2p=max_x-((x2*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y2p=min_y+((x2*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x3p=max_x-((x2*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y3p=min_y+((x2*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    x4p=max_x-((x1*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y4p=min_y+((x1*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    return Polygon(((y1p,x1p),(y2p,x2p),(y3p,x3p),(y4p,x4p),(y1p,x1p)))

def map_d(x, in_min, in_max, out_min, out_max):
    """Genera una interpolacion para pasar de un rango a otro"""
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def postproceso(Modelo,model_class,casas,conf_casas,clase_casas,terreno,conf_terreno,clase_terreno,raster,ancho,alto,dim,minx,maxx,miny,maxy,shape,angulo_get=0,opt_conf_thres=0.05,imshow=False,imsave=False,path=""):
    with rasterio.open(raster) as src:
        with tqdm.tqdm(total=alto*ancho) as pbar:
            for j in range(ancho):#ancho
                for i in range(alto):#alto
                    generar=0
                    label=raster.replace('\\','/').split('/')[-1][:-4]+'_'
                    nameimg=label.lower()+str(i)+'_'+str(j)
                    cuadro=[]
                    for k in range(2):
                        for l in range(2):
                            cuadro.append((minx+(maxx-minx)/ancho*(j+k),maxy-(maxy-miny)/alto*(i+l)))
                    cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                    cvees=[]
                    for punto in cuadro:
                        x=float(punto[0])
                        y=float(punto[1])
                        mini_df=shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)]
                        if len(mini_df)>0:
                            generar=1
                            cvees.append(mini_df["cve_cat"].values)### traer todas las cve_catastrales del punto sobre el raster ...Pendiente
#                             print(cvees)
                    if generar==1:
                        shapes=[{'type':'Polygon','coordinates':[cuadro]}]
                        vector=[]
                        array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                        if np.sum(array)<100:
                            pbar.update(1)
                            continue
                        four_images=[array[2],array[1],array[0]]
                        imagen_n = np.stack(four_images, axis=-1)
                        if imsave:
                            cv2.imwrite(path+"/"+nameimg+".png",imagen_n)
                            # print(path+"/"+nameimg+".png")
                        if angulo_get!=0:
                            angulo=-angulo_get
                        else:
                            angulo=correct_orientation(imagen_n,dim=dim)[0]
                        image_ro=imagen_n.copy()
                        image_ro=rotate_image(image_ro,angulo,reshape=True)
                        w=image_ro.shape[0]
                        h=image_ro.shape[1]
                        with torch.no_grad():
                            vector=Modelo.detect(opt_source="cache1.png",opt_conf_thres=opt_conf_thres,imagen_s=image_ro)
                        proyecciones=shapes[0].get('coordinates')[0][:-1]
                        df_cache=vector2xy(vector,w,h,dim=dim,nameimg=nameimg)
                        for cs_1 in (range(len(df_cache))):
    #                         x1,y1=df_cache.loc[cs_1,'start_point_im']
    #                         x2,y2=df_cache.loc[cs_1,'end_point_im']
    #                         df_aux=image_ro.copy()
    #                         df_aux=df_aux[y1:y2,x1:x2]
    #                         clase,imagen=model_class.predict_iamge(df_aux)
    #                         if np.sum(df_aux)>500:
    #                             if crear:
    #                                 name=r"C:\Users\ASUS\Inteligencia_Artificial\calsificador\salida/"+clase+"/"+str(datetime.datetime.now()).replace(" ","_").replace(":","_").replace(".","_").replace("-","_")+".png"
    #                                 cv2.imwrite(name,imagen)
                            x1,y1=df_cache.loc[cs_1,'start_point_im']
                            x2,y2=df_cache.loc[cs_1,'end_point_im']
                            df_aux=image_ro.copy()
                            df_aux=df_aux[y1:y2,x1:x2]
                            clase,imagen=model_class.predict_image(df_aux)
                            if df_cache['Tipo'][cs_1]=='casa':
                                if np.sum(df_aux)>500:
                                    casas.append(rotacion_detect(df_cache.loc[cs_1,'start_point_100'], df_cache.loc[cs_1,'end_point_100'],-angulo,proyecciones,w,h,dim))
                                    conf_casas.append(df_cache.loc[cs_1,'conf'])
                                    clase_casas.append(clase)

                            else:
                                terreno.append(rotacion_detect(df_cache.loc[cs_1,'start_point_100'], df_cache.loc[cs_1,'end_point_100'],-angulo,proyecciones,w,h,dim))
                                conf_terreno.append(df_cache.loc[cs_1,'conf'])
                                clase_terreno.append(clase)
                        if imshow:
                            print(angulo)
                            imshow_detect(df_cache,image_ro)

                    pbar.update(1)
                    
def Parametro_raster(raster,metros=120):
    """Se obtienen las dimensiones del raster y del cuadro de corte"""
    gdal_interpeter = gdal.Open(raster)
    width = gdal_interpeter.RasterXSize
    height = gdal_interpeter.RasterYSize
    coordenadas_gdal = gdal_interpeter.GetGeoTransform()
    minx = coordenadas_gdal[0]
    miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
    maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
    maxy = coordenadas_gdal[3]
    src_raster_path = raster
    src=rasterio.open(src_raster_path)
    H,W=src.shape
    dim=int(np.ceil(map_d(minx+metros,minx,maxx,0,W)))
    alto=np.max([1,int(np.floor(H/dim))])
    ancho=np.max([1,int(np.floor(W/dim))])
    return alto,ancho,dim,src.crs,H,W,minx,maxx,miny,maxy


def shape_transform(shape):
    """Convierte el shape en dataframe de coordenadas que engloba el polygon del shape para delimitar el raster"""
    c=[]
    angulo_manzana=[]

    # for manzana in range(len(shape)):
    #     proyecciones1=mapping(shape['geometry'][manzana]).get('coordinates')
    #     angulos=[]
    #     d=[]
    #     poly=pd.DataFrame(proyecciones1[0])
    #     for point in range(1,len(poly)):
    #         d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
    #         angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
    #     angulo_manzana.append(angulos[d.index(max(d))])
    shape["angulo_manzana"]=0
    shape["geometry"]=shape["geometry"].envelope
    for manzana in range(len(shape)):
        proyecciones1=mapping(shape['geometry'][manzana]).get('coordinates')
        proyecciones=proyecciones1[0]
        point1=np.min((proyecciones,proyecciones),axis=1)[0]
        min_y,min_x=point1[0],point1[1]
        point2=np.max((proyecciones,proyecciones),axis=1)[0]
        max_y,max_x=point2[0],point2[1]
        c.append(','.join([str(min_y),str(min_x),str(max_y),str(max_x)]))
    shape1=pd.DataFrame()
    shape1['points']=c
    shape1=shape1['points'].str.split(',',expand=True)
    shape1=shape1.astype({0:'float64',1:'float64',2:'float64',3:'float64'})
    shape1["cve_cat"]=shape["cve_cat"]
    shape1["angulo_manzana"]=shape["angulo_manzana"]
    shape=shape1
    return shape
    
def ampliar_shape(shape,factor_ampliacion=2):
    """Amplifica el polygon de cada manzana con el fin de extrar imagenes sin perder informacion de la manzana"""
    shape["geometry"]=shape["geometry"].envelope
    shape['centroid']=shape.centroid
    geometry=[]
    for i,polygon in enumerate(shape['geometry']):
        point=mapping(shape['centroid'][i]).get('coordinates')
        x=point[0]
        y=point[1]
        go=[]
        coodinates=mapping(polygon).get('coordinates')[0]
        for a in coodinates:
            x1=a[0]
            y1=a[1]
            x2=x+(x1-x)*factor_ampliacion
            y2=y+(y1-y)*factor_ampliacion
            go.append((x2,y2))
        geometry.append(Polygon(go))
    return gpd.GeoDataFrame(shape["cve_cat"],geometry=geometry)

# idx_to_class={0: 'area_verde', 1: 'carros', 2: 'casas', 3: 'en_construccion', 4: 'establecimiento', 5: 'multivivienda', 6: 'terreno_baldio'}
class alexnet():
    def __init__(self,weights,num_classes,idx_to_class):
        """inicializa el model, con los pesos entrenados"""
        alexnet=models.alexnet(pretrained=True)
        self.device = torch.device(0 if torch.cuda.is_available() else "cpu")
        checkpoint=torch.load(weights,map_location=self.device)
        # alexnet.features[1]= nn.Hardtanh()
        alexnet.classifier[6] = nn.Linear(4096, num_classes)
        alexnet.classifier.add_module("7", nn.LogSoftmax(dim = 1))
        summary(alexnet, (3, 224, 224))
        self.model=alexnet
        self.idx_to_class=idx_to_class
    
    def predict_file(self,file,pad=True):
        """Genera prediccion sobre archivo"""
        # x = Image.open(file)
        # x = np.asarray(x)
        # x=np.stack([x[:,:,0],x[:,:,1],x[:,:,2]], axis=-1)
        x=cv2.imread(file)
        if pad:
            x=padding(x)
        x=cv2.resize(x,(224,224))
        x=x.astype("float32")
        x=x/255*2-1
        x=np.moveaxis(x,-1,0)
        x = np.expand_dims(x, axis=0)
        with torch.no_grad():
            img = torch.from_numpy(x).to(self.device)
            res=list(self.model(img).cpu().detach().numpy()[0])
            indice=res.index(max(res))
            clase=self.idx_to_class.get(indice)
        return clase 
    
    def predict_image(self,image,pad=True):
        """Generar predeccion de clase sobre imagen precargada"""
        x = np.array(image)
        # x=np.stack([x[:,:,0],x[:,:,1],x[:,:,2]], axis=-1)
        if pad:
            x=padding(x)
        imagen=x.copy()
        x=cv2.resize(x,(224,224))
        x=x.astype("float32")
        x=x/255*2-1
        x=np.moveaxis(x,-1,0)
        x = np.expand_dims(x, axis=0)
        with torch.no_grad():
            img = torch.from_numpy(x).to(self.device)
            res=list(self.model(img).cpu().detach().numpy()[0])
            indice=res.index(max(res))
            clase=self.idx_to_class.get(indice)
        return clase, imagen
    
def padding(img):
    """Escala la imagen y completa el sobrante con franjas negras, para no perder proporciones"""                
    old_image_height, old_image_width, channels = img.shape
    new_image_width = 224
    new_image_height = 224
    color = (0,0,0)
    if old_image_height<=old_image_width:
        f=new_image_width/old_image_width
    else:
        f=new_image_height/old_image_height
    img=cv2.resize(img,(int(f*old_image_width),int(f*old_image_height)))
    old_image_height, old_image_width, channels = img.shape
    result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)
    x_center = (new_image_width - old_image_width) // 2
    y_center = (new_image_height - old_image_height) // 2
    try:
        result[y_center:y_center+old_image_height, 
            x_center:x_center+old_image_width] = img
    except:
        result=img 
    return result        
class mobilenet_class():
    def __init__(self,alexnet,weights,num_classes,idx_to_class):
        """inicializa el model, con los pesos entrenados"""
        # alexnet=models.alexnet(pretrained=True)
        self.device = torch.device(0 if torch.cuda.is_available() else "cpu")
        
        checkpoint=torch.load(weights,map_location=self.device)
        # checkpoint=torch.load(weights)
        # 
        alexnet.classifier[3] = nn.Linear(1024, num_classes)
        alexnet.classifier.add_module("4", nn.LogSoftmax(dim = 1))
        # for param in alexnet.parameters():
        #     param.requires_grad = False
        # alexnet.classifier[6] = nn.Linear(4096, num_classes)
        # alexnet.classifier.add_module("7", nn.LogSoftmax(dim = 1))
        alexnet.load_state_dict(checkpoint['model_state_dict'])
        summary(alexnet.to(self.device), (3, 224, 224))
        self.model=alexnet
        self.idx_to_class=idx_to_class

    
    def predict_file(self,file,pad=True):
        """Genera prediccion sobre archivo"""
        x = Image.open(file)
        x = np.asarray(x)
        # if pad:
        #     # resize
        #     x= x.resize()
        x=cv2.resize(x,(224,224))
        x=x.astype("float32")
        # Cambiar normalizacion
        x=x/255*2-1

        x=np.moveaxis(x,-1,0)
        x = np.expand_dims(x, axis=0)
        with torch.no_grad():
            img = torch.from_numpy(x).to(self.device)
            res=list(self.model(img).cpu().detach().numpy()[0])
            indice=res.index(max(res))
            clase=self.idx_to_class.get(indice)
        return clase 
    
    def predict_image(self,image,pad=True):
        """Generar predeccion de clase sobre imagen precargada"""
        x = np.asarray(image)
        # if pad:
        #     x=padding(x)
        imagen=x.copy()
        x=cv2.resize(x,(224,224))
        x=x.astype("float32")
        # Cambiar normalizacion
        x=x/255*2-1
        x=np.moveaxis(x,-1,0)
        x = np.expand_dims(x, axis=0)
        with torch.no_grad():
            img = torch.from_numpy(x).to(self.device)
            res=list(self.model(img).cpu().detach().numpy()[0])
            indice=res.index(max(res))
            clase=self.idx_to_class.get(indice)
        return clase, imagen 

In [121]:
def matchingTemplate(img_rgb,template,threshold = .7):
    '''
(Function)
(Parameters)
    - threshold: Porcentaje de coincidencia 
    '''
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    loc = np.where( res >=threshold)
    startpoints=[]
    endpoints=[]
    thresh=[]
    x=-10
    y=-10
    for i,pt in enumerate(zip(*loc[::-1])):
        #if i==0:
        x=pt[0]
        y=pt[1]
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        cv2.putText(img_rgb,str(i) ,(x,y) , cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        startpoints.append(pt)
        endpoints.append((pt[0] + w, pt[1] + h))
        val=np.array(loc)[:,i]
        thresh.append(res[val[0],val[1]])
        if (pt[0]!=x and pt[0]+1!=x and pt[0]-1!=x and pt[0]+2!=x and pt[0]-2!=x) and (pt[1]!=y and pt[1]+1!=y and pt[1]-1!=y and pt[1]+2!=y and pt[1]-2!=y):
            x=pt[0]
            y=pt[1]
            cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
            cv2.putText(img_rgb,str(i) ,(x,y) , cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
            startpoints.append(pt)
            endpoints.append((pt[0] + w, pt[1] + h))
            val=np.array(loc)[:,i]
            thresh.append(res[val[0],val[1]])
    return img_rgb,startpoints,endpoints,thresh

In [109]:
raster=r"C:\Users\ruben\Downloads\Chinchetas_Ixtapan_centro.tif"
metros = 100
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=metros)
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy

(23,
 43,
 286,
 CRS.from_wkt('LOCAL_CS["WGS 84 / Pseudo-Mercator",UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 6774,
 12347,
 -11098076.2292,
 -11093754.7532,
 2135322.7351,
 2137693.5448)

In [58]:
6*dim/100

60.06

In [74]:
factor

28.6

In [110]:
template = cv2.imread(r"C:\Users\ruben\Desktop\GEM\Geo\Alexnet\nuevo.png")
template.shape
factor = ((6*dim/metros)/21)
temp = cv2.resize(template,(int((factor)*template.shape[1]),int((factor)*template.shape[0])))
# pd.DataFrame(temp[:,:,0]).to_clipboard()

In [5]:
shape_o=gpd.read_file(r"C:\Users\ruben\Desktop\GEM\Geo\Bases\Full manzanas\Manzana_Ixtapan_Sal.shp")
shape_o=shape_o.to_crs('3857')
shape=shape_transform(shape_o.copy())
#shape_aumentado=ampliar_shape(shape_o.copy())

In [6]:
def rotacion_detect(startpoint,endpoint,angle,proyecciones,w,h,dim):
    """Rotata el cuadro detectado en el sistema de coordenadas inicial"""
    point1=np.min((proyecciones,proyecciones),axis=1)[0]
    min_y,min_x=point1[0],point1[1]
    point2=np.max((proyecciones,proyecciones),axis=1)[0]
    max_y,max_x=point2[0],point2[1]
    min_y,min_x,max_y,max_x,proyecciones
    tipos=["casa","terreno"]
    y1,x1=startpoint
    y2,x2=endpoint
    x1,y1=(x1)*2-1,(y1)*2-1
    x2,y2=(x2)*2-1,(y2)*2-1
    angle=angle*math.pi/180
#     x1,y1=x1*(w/dim),y1*(h/dim)
#     x2,y2=x2*(w/dim),y2*(h/dim)
    #x_p, y_p son los puntos de un rectangulo en el orden inverso al manecillas del reloj
    x1p=max_x-((x1*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y1p=min_y+((x1*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x2p=max_x-((x2*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y2p=min_y+((x2*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x3p=max_x-((x2*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y3p=min_y+((x2*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    x4p=max_x-((x1*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y4p=min_y+((x1*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    return Polygon(((y1p,x1p),(y2p,x2p),(y3p,x3p),(y4p,x4p),(y1p,x1p)))

In [7]:
diccionario = {0: 'Hospital', 1: 'Jardin', 2: 'a_verdes', 3: 'a_verdes_1', 4: 'a_verdes_2', 5: 'bancos_google', 6: 'bar',
               7: 'cine', 8: 'correo', 9: 'cultural_3', 10: 'dinero', 11: 'educativo', 12: 'entretenimiento_google', 
               13: 'establecimientos_2_google', 14: 'establecimientos_google', 15: 'gas_google', 16: 'gobierno_google', 
               17: 'golf', 18: 'hospedaje_google', 19: 'juegos_google', 20: 'playa_google', 21: 'poli', 22: 'restaurante_google',
               23: 'salud_google', 24: 'sitios_interes_google', 25: 'templo_google', 26: 'tiendas_1', 27: 'tiendas_2', 28: 'turista_google'}
weights = r"C:\Users\ruben\Desktop\GEM\Geo\Alexnet\Modelos\best_mod1_pines.pth"
# modelo = models.mobilenet_v3_small(weights='DEFAULT')
mobilenet = alexnet(weights,len(diccionario),diccionario)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 256, 6, 6]           --
|    └─Conv2d: 2-1                       [-1, 64, 55, 55]          23,296
|    └─ReLU: 2-2                         [-1, 64, 55, 55]          --
|    └─MaxPool2d: 2-3                    [-1, 64, 27, 27]          --
|    └─Conv2d: 2-4                       [-1, 192, 27, 27]         307,392
|    └─ReLU: 2-5                         [-1, 192, 27, 27]         --
|    └─MaxPool2d: 2-6                    [-1, 192, 13, 13]         --
|    └─Conv2d: 2-7                       [-1, 384, 13, 13]         663,936
|    └─ReLU: 2-8                         [-1, 384, 13, 13]         --
|    └─Conv2d: 2-9                       [-1, 256, 13, 13]         884,992
|    └─ReLU: 2-10                        [-1, 256, 13, 13]         --
|    └─Conv2d: 2-11                      [-1, 256, 13, 13]         590,080
|    └─ReLU: 2-12                        [-1, 256, 13, 13]   

In [8]:
dict_gris = get_dict_plantilla_gris(r"C:\Users\ruben\Downloads\Plantillas-20230512T162122Z-001\Plantillas")
detectar_clase(r"C:\Users\ruben\Desktop\GEM\Geo\Alexnet\nuevo.png",dict_gris,True)

[0]

In [122]:
# from google.colab.patches import cv2_imshow

chincheta=[]
clases=[]
threshold=[]
show_imagen = False
# Plantilla de reconociemientt
# template = cv2.imread(r"C:\Users\ruben\Desktop\GEM\Geo\Alexnet\nuevo.png")
template = temp
dict_gris = get_dict_plantilla_gris(r"C:\Users\ruben\Downloads\Plantillas-20230512T162122Z-001\Plantillas")
template_gray=template[:,:,1]
with rasterio.open(raster) as src:
    with tqdm.tqdm(total=alto*ancho*4) as pbar:
        for j in range(ancho*2):#ancho
            for i in range(alto*2):#alto
                generar=0
                label=raster.replace('\\','/').split('/')[-1][:-4]+'_'
                nameimg=label.lower()+str(i)+'_'+str(j)
                cuadro=[]
                for k in range(2):
                    for l in range(2):
                        cuadro.append((minx+(maxx-minx)/ancho*(j/2+k),maxy-(maxy-miny)/alto*(i/2+l)))
                cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                for punto in cuadro:
                    x=float(punto[0])
                    y=float(punto[1])
                    mini_df=shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)]
                    if len(mini_df)>0:
                        generar=1
                        #cvees.append(mini_df["cve_cat"].values)### traer todas las cve_catastrales del punto sobre el raster ...Pendiente
#                             print(cvees)
                if generar==1:                    
                    shapes=[{'type':'Polygon','coordinates':[cuadro]}]
                    vector=[]
                    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                    if np.sum(array)<100:                        
                        pbar.update(1)
                        continue
                    four_images=[array[2],array[1],array[0]]
                    imagen_n = np.stack(four_images, axis=-1)
                    image_o=imagen_n.copy()
                    try:                                               
                        imagen_n,startpoints, endpoints, thresh=matchingTemplate(imagen_n,template_gray,threshold = .7)                        
                    except Exception as e:
                        # print(e)                                              
                        pbar.update(1)
                        continue
                    
                    for s in range(len(startpoints)):
                        startpoint=startpoints[s]                        
                        endpoint=endpoints[s]
                        df_aux=image_o.copy()
                        x1,y1=startpoint
                        x2,y2=endpoint
                        df_aux=df_aux[y1:y2,x1:x2] 
                        # clasificacion

                        clase_aux  = detectar_clase(df_aux, dict_gris)
                        clases.append(clase_aux)
                        threshold.append(thresh[s])
                        # En caso de querer almacenar lo cuadritos, poner ruta
    #                     cv2.imwrite(r"C:\Users\ASUS\Inteligencia_Artificial\pines/"+str(j)+"_"+str(i)+"_"+str(s)+"deteccion.png",df_aux)

                        startpoint=[x/dim for x in startpoint]
                        endpoint=[x/dim for x in endpoint]
                        proyecciones=cuadro
                        chincheta.append(rotacion_detect(startpoint,endpoint,0,proyecciones,dim,dim,dim))
                    if show_imagen:
                        cv2.imshow('1',imagen_n)
                        cv2.waitKey()
                        cv2.destroyAllWindows()

                
                
                pbar.update(1)
                
                

100%|██████████| 3956/3956 [01:49<00:00, 36.15it/s]


In [120]:
gps=gpd.GeoDataFrame({'Clase':clases,'Threshold':threshold}, geometry=chincheta,crs=3857)
# gps["centroidx"]=gps.centroid.x/10
# gps["centroidy"]=gps.centroid.y/10
# gps["centroidx"]=gps["centroidx"].astype("int")
# gps["centroidy"]=gps["centroidy"].astype("int")
# gps=gps.drop_duplicates(["centroidx","centroidy"])
gps.to_file(r"C:\Users\ruben\Downloads\Ixtapan_centro_chinchetas.shp")

In [37]:
threshold

[[0.51551586, 0.50195074],
 [0.51551586, 0.50195074],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.51446337],
 [0.50252914, 0.50569636, 0.5498868],
 [0.50252914, 0.50569636, 0.5498868],
 [0.50252914, 0.50569

In [29]:
len(chincheta), len(clases), len(threshold)

(41023, 41023, 41023)

[[0.51551586, 0.50195074],
 [0.51551586, 0.50195074],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5155002, 0.5019246, 0.50228417, 0.50485873, 0.5061941, 0.5020283],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.5022672, 0.5048567, 0.50619, 0.5000227, 0.5020283, 0.51446337],
 [0.51446337],
 [0.50252914, 0.50569636, 0.5498868],
 [0.50252914, 0.50569636, 0.5498868],
 [0.50252914, 0.50569

In [ ]:
def cv2_imshow(name,image):
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name,image)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [ ]:
import glob
filenames=glob.glob(r"C:\Users\ASUS\Inteligencia_Artificial\pines/*.png")

In [ ]:
import tqdm
file='C:\\Users\\ASUS\\Inteligencia_Artificial\\chinchetas\\0_28_1deteccion.png'
valor=[]
color=[]
for file in tqdm.tqdm(filenames):
    image2=cv2.imread(file)
    Z = image2.reshape((-1,3))
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 1
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    # Now convert back into uint8, and make original image
    center = np.uint8(center[0])
    color.append("".join([str(int(np.ceil(x/255*10))).zfill(3) for x in center]))
    _,image=cv2.threshold(image2[:,:,1],200,255,cv2.THRESH_BINARY)
    # image=image[5:41,6:29]
    image=image/255
    images=pd.DataFrame(image)
    x1=0
    x2=100
    for col in images.columns:
        if images[col].sum()>=len(images)-5:
            if int(col)>=x1 and int(col)<=int(len(images.columns)/2):
                x1=int(col)+1
            if int(col)>=x1 and int(col)<=x2 and int(col)>=int(len(images.columns)/2):
                x2=int(col)
    y1,y2=(0,len(images))
    image1=image[y1:y2,x1:x2]
    images=pd.DataFrame(image1)
    y1=0
    y2=100
    for i in range(len(images)):
        if images.iloc[i].sum()>=len(images.columns)-5:
            if int(i)>=y1 and int(i)<=int(len(images)/2):
    #             print(y1)
                y1=int(i)+1
            if int(i)>=y1 and int(i)<=y2 and int(i)>=int(len(images)/2):
                y2=int(i)
    image1=image1[y1:y2,0:len(images.columns)]
#     images=pd.DataFrame(image1)
#     x1=0
#     x2=100
#     for col in images.columns:
#         if images[col].sum()==len(images):
#             if int(col)>=x1 and int(col)<=int(len(images.columns)/2):
#                 x1=int(col)+1
#             if int(col)>=x1 and int(col)<=x2 and int(col)>=int(len(images.columns)/2):
#                 x2=int(col)
#     y1,y2=(0,len(images))
#     image1=image1[y1:y2,x1:x2]
#     cv2_imshow("a",image1)
    valor.append(np.sum(image1))
res=pd.DataFrame({"file":filenames,"valor":valor,"color":color})
res

In [ ]:
res["valor1"]=[int(x/10) for x in res["valor"]]
res["valor2"]=[int(str(x).replace(".","")) for x in res["color"]]
Z = np.vstack((res["valor"],res["valor2"]))
# convert to np.float32
Z = np.float32(Z)
# define criteria and apply kmeans()
# criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# ret,label,center=cv.kmeans(Z,2,None,criteria,10,cv.KMEANS_RANDOM_CENTERS)
# res.groupby("valor2").count()
import matplotlib.pyplot as plt
plt.scatter(res["valor1"],res["valor2"],cmap='viridis',marker="o")
# A = Z[label.ravel()==0]
# B = Z[label.ravel()==1]
# # Plot the data
# plt.scatter(A[:,0],A[:,1])
# plt.scatter(B[:,0],B[:,1],c = 'r')
# plt.scatter(center[:,0],center[:,1],s = 80,c = 'y', marker = 's')
# plt.xlabel('Height'),plt.ylabel('Weight')
# plt.show()

In [ ]:
res["valor2"].value_counts()

In [ ]:
aux=res[res["valor2"]==9009008].sort_values(["valor1","valor2"]).reset_index(drop=True).head(100)
aux

In [ ]:
for i in range(len(aux)):
    im=cv2.imread(aux["file"][i])
    cv2_imshow(str(aux.loc[i,"valor1"])+"_"+str(aux.loc[i,"valor2"]),im)